# Цель работы:
получить список рекомендаций для указанного пользователя, например:

Входные данные: идентификатор клиента  

Результат: ранжированный список товаров (идентификаторов продуктов), которые пользователь, скорее всего, захочет положить в свою (пустую) «корзину».  

In [1]:
import pandas as pd
import numpy as np

from sklearn.decomposition import TruncatedSVD
from sklearn.neighbors import NearestNeighbors
from collections import defaultdict, Counter

from surprise import Dataset, Reader
from surprise.prediction_algorithms.matrix_factorization import SVD
from surprise import accuracy
from surprise.prediction_algorithms.knns import KNNBasic

# Анализ данных

Чтение данных из файлов.

In [3]:
customers = pd.read_csv('data/recommend_1.csv')
transactions = pd.read_csv('data/trx_data.csv')

Рассмотрим состав и размерность датасетов.

In [4]:
print(customers.shape)
customers.head()

(1000, 1)


,customerId
0,1553
1,20400
2,19750
3,6334
4,27773


In [5]:
len(customers['customerId'].unique())

1000

In [6]:
print(transactions.shape)
transactions.head()

(62483, 2)


,customerId,products
0,0,20
1,1,2|2|23|68|68|111|29|86|107|152
2,2,111|107|29|11|11|11|33|23
3,3,164|227
4,5,2|2


In [7]:
len(transactions['customerId'].unique())

24429

Число уникальных идентификаторов клиентов в датасете 'transactions' больше чем в 'customers'. В дальнейшем будем работать только с датасетом 'transactions', как содержащем более полную информацию.

# Преобразование
В столбце 'products' датасета 'transactions' купленные продукты перечислены через разделитель '|'. Необходимо каждый продукт записать в отдельную строку. Также добавим столбец 'purchase_count' с указанием количества раз которое данный продукт купил конкретный клиент.

In [8]:
transactions['products'] = transactions['products'].apply(lambda x: [int(i) for i in x.split('|')])

data = pd.melt(transactions.set_index('customerId')['products'].apply(pd.Series).reset_index(), 
             id_vars=['customerId'],
             value_name='products') \
    .dropna().drop(['variable'], axis=1) \
    .groupby(['customerId', 'products']) \
    .agg({'products': 'count'}) \
    .rename(columns={'products': 'purchase_count'}) \
    .reset_index() \
    .rename(columns={'products': 'productId'})
data['productId'] = data['productId'].astype(np.int64)

In [9]:
print(data.shape)
data.head()

(133585, 3)


,customerId,productId,purchase_count
0,0,1,2
1,0,13,1
2,0,19,3
3,0,20,1
4,0,31,2


Построим сводную таблицу покупок для каждого клиента. Строки-идентификатор клиента. Столбцы - продукты.

In [10]:
df_matrix = pd.pivot_table(data, values='purchase_count', index='customerId', columns='productId')
df_matrix.head()

productId,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
customerId,,,,,,,,,,,,,,,,,,,,,
0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
df_matrix.fillna(0, inplace=True)
df_matrix.head()

productId,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
customerId,,,,,,,,,,,,,,,,,,,,,
0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
df_matrix.shape

(24429, 300)

# Вариант 1 рекомендательной системы. 
Построим рекомендательную систему на основе kNN (k ближайших соседей) используя модуль sklearn.neighbors.NearestNeighbors с алгоритмом 'brute'.  
Обучим модель по полным данным df_matrix.  

In [15]:
knn = NearestNeighbors(n_neighbors=10, algorithm= 'brute', metric= 'cosine')
model_knn = knn.fit(df_matrix)

Напишем функцию, которая в зависимости от идентификатора клиента возвращает 10 наиболее похожих по покупкам клиентов.  

In [16]:
def most_similar_users_to(user_id):
    most_similar_users_to = []
    distance, indice = model_knn.kneighbors(df_matrix.iloc[user_id,:].values.reshape(1,-1), n_neighbors=10)
    print('Рекомендации для ##клиента с идентификатором {0} ##:'.format(df_matrix.index[user_id]))
    for i in range(1, len(distance.flatten())):
        user_id1 = df_matrix.index[indice.flatten()[i]]
        most_similar_users_to.append((user_id1, distance.flatten()[i]))

    most_similar_users_to.sort(key=lambda x: x[1], reverse=True)

    return most_similar_users_to[:10]   

In [17]:
print("Подобие (схожесть) на основе пользователя")
print(most_similar_users_to(4))

Подобие (схожесть) на основе пользователя
Рекомендации для ##клиента с идентификатором 4 ##:
[(25368, 0.3675444679663241), (17524, 0.3675444679663241), (14285, 0.3675444679663241), (27284, 0.3675444679663241), (23543, 0.3675444679663241), (23715, 0.3675444679663241), (25477, 0.3675444679663241), (165, 0.35949714876589), (5846, 0.3545027756320972)]


Напишем функцию рекомендации продукта исходя из покупок клиентов с наиболее похожим поведением.  

In [18]:
def user_based_suggestions(user_id):
    # суммировать все коэффициенты подобия
    suggestions = defaultdict(float)
    non_interacted_shopping = df_matrix.iloc[user_id][df_matrix.iloc[user_id]==0].index.tolist()
    for other_user_id, similarity in most_similar_users_to(user_id):
        shopping_user_id = df_matrix.loc[other_user_id][df_matrix.loc[other_user_id]>0]
        for interest in shopping_user_id.index.tolist():
            if interest in non_interacted_shopping:
                suggestions[interest] += similarity

    # преобразовать их в сортированный список
    suggestions = sorted(suggestions.items(),
                         key=lambda x: x[1],
                         reverse=True)
    return suggestions[:10]

Получим рекомендации для клиентов с идентификаторами 4 и 21.

In [19]:
print("Рекомендации для пользователя")
print(user_based_suggestions(4))

Рекомендации для пользователя
Рекомендации для ##клиента с идентификатором 4 ##:
[(1, 0.7139999243979872), (5, 0.7139999243979872), (7, 0.35949714876589), (25, 0.35949714876589), (31, 0.35949714876589), (33, 0.35949714876589), (52, 0.35949714876589), (57, 0.35949714876589), (61, 0.35949714876589), (87, 0.35949714876589)]


In [29]:
print("Рекомендации для пользователя")
print(user_based_suggestions(21))

Рекомендации для пользователя
Рекомендации для ##клиента с идентификатором 21 ##:
[(1, 0.364224468608779), (38, 0.364224468608779), (142, 0.364224468608779), (179, 0.364224468608779), (273, 0.364224468608779)]


# Вариант 2 рекомендательной системы.  
Построим рекомендательную систему на основе kNN (k ближайших соседей) используя модуль surprise.prediction_algorithms.knns с мерой подобия 'cosine'.

Для работы с surprise.prediction_algorithms.knns требуется датасет 'data' преобразовать в trainset (данные более высокого уровня).

In [20]:
reader = Reader(line_format='user item rating', sep=',', rating_scale=(0,5), skip_lines=1)
data1 = Dataset.load_from_df(data[['customerId','productId','purchase_count']].iloc[:,:], reader=reader)
trainset = data1.build_full_trainset()

In [21]:
%%time
# Параметры сходства
sim_options = {'name': 'cosine',
               'user_based': True}

# KNN 
sim_user = KNNBasic(sim_options=sim_options, verbose=False, random_state=33)

sim_user.fit(trainset)

CPU times: total: 2min
Wall time: 2min 21s


Напишем функцию возвращиющую список рекомендаций в зависимости от идентификатора клиента.

In [26]:
def get_recommendations(data, user_id, top_n, algo):
    
    recommendations = []
    
    user_shopping_interactions_matrix = data.pivot(index='customerId', columns='productId', values='purchase_count')
    
    non_interacted_shopping = user_shopping_interactions_matrix.loc[user_id][user_shopping_interactions_matrix.loc[user_id].isnull()].index.tolist()
    
    for item_id in non_interacted_shopping:
        
        est = algo.predict(user_id, item_id).est
        
        recommendations.append((item_id, est))

    recommendations.sort(key=lambda x: x[1], reverse=True)

    return recommendations[:top_n]

Получим рекомендации для клиентов с идентификаторами 4 и 21.

In [27]:
get_recommendations(data, 4,10,sim_user)

[(132, 4.030769230769231),
 (37, 3.65),
 (82, 3.65),
 (34, 3.3),
 (0, 3.275),
 (252, 3.057341422640977),
 (248, 3.0501576825090466),
 (110, 2.95),
 (73, 2.937674214867727),
 (68, 2.725)]

In [28]:
get_recommendations(data, 21,10,sim_user)

[(252, 3.7077892272250033),
 (37, 3.55),
 (34, 3.475),
 (82, 3.45),
 (0, 3.275),
 (248, 3.2524942305383813),
 (132, 3.046851879863488),
 (110, 2.975),
 (68, 2.875),
 (27, 2.8254169614663662)]

# Выводы.
Построено две рекомендательные системы подбора товаров по идентификатору клиента на основе kNN (k ближайших соседей) используя модули:
- sklearn.neighbors.NearestNeighbors с алгоритмом 'brute'; 
- surprise.prediction_algorithms.knns с мерой подобия 'cosine'.